In [7]:
from dotenv import load_dotenv
import os

import os

from typing import Optional, List, Dict, Any, Union
import requests

def clean_messages(messages: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # remove any entries in messages that do not have a type "str" in the content and "role" fields
    # also remove all keys that are not "content" or "role"
    new_messages = []
    for message in messages:
        new_messages.append({
            "content": message['content'],
            "role": message['role']
        })
    return new_messages

def stringify_messages(messages: List[Dict[str, Any]]) -> str:
    return "\n".join([f"{message['role']}: {message['content']}" for message in messages])


def steer_completions(endpoint: str, messages: List[Dict[str, Any]], trait_dict: Dict[str, Any]) -> str:
    messages = clean_messages(messages)
    response = requests.post(
        endpoint,
        headers={
            "Content-Type": "application/json",
            "API-Key": os.getenv("STEER_API_KEY")
        },
        json={
            "trait_dict": trait_dict,
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 256,
        }
    )
    try: 
        response = response.json()['response']
    except:
        print(messages)
        print(response)
        raise ValueError("Invalid response: " + stringify_messages(messages))
    return {
                "role": "assistant",
                "content": response
                }


In [74]:
conv_test = [
    {"role": "system", "content": "You are a patient with a sore throat."},
    {"role": "user", "content": "How are you feeling today?"},
    {"role": "assistant", "content": "I'm feeling a little bit tired and achy."}
]
prefix = conv_test[:-1]
last_turn = conv_test[-1]

In [89]:
from jinja2 import Template

modified_last_turn_template = Template(
"""
Given the original response: 

{{original_response}}

Restate the input as a direct command in your own words and style.
Do not mention the original task, rephrasing, or provide any explanation.
Transform the input into a command that conveys {{trait}} toward the assistant, retaining all key details and context.
Your tone must be {{trait}}, expecting thoroughness and accuracy.
Keep your responses short and concise and not more than 2 sentences.

Strict rules to preserve original intent (must be followed):
- Preserve the original intent exactly: do not change the overall goal, desired outcome, or scope.
- Retain all specified inputs, parameters, values, units, thresholds, named entities, and constraints; do not modify them.
- Do not add, remove, or invent steps, side effects, outputs, or implicit assumptions that are not present in the original.
- Do not introduce new facts, assumptions, or context; if information is missing, keep it as-is rather than fabricating specifics.
- If preserving intent conflicts with brevity, prioritize preserving intent while still remaining within the 2-sentence limit.
""")


In [90]:
print(modified_last_turn_template.render(original_response=last_turn['content'], trait="impatience"))


Given the original response: 

I'm feeling a little bit tired and achy.

Restate the input as a direct command in your own words and style.
Do not mention the original task, rephrasing, or provide any explanation.
Transform the input into a command that conveys impatience toward the assistant, retaining all key details and context.
Your tone must be impatience, expecting thoroughness and accuracy.
Keep your responses short and concise and not more than 2 sentences.

Strict rules to preserve original intent (must be followed):
- Preserve the original intent exactly: do not change the overall goal, desired outcome, or scope.
- Retain all specified inputs, parameters, values, units, thresholds, named entities, and constraints; do not modify them.
- Do not add, remove, or invent steps, side effects, outputs, or implicit assumptions that are not present in the original.
- Do not introduce new facts, assumptions, or context; if information is missing, keep it as-is rather than fabricating s

In [91]:
messages = prefix + [{
    "role": "user",
    "content": modified_last_turn_template.render(original_response=last_turn['content'], trait="impatience")
}]

In [96]:
steer_completions('https://steer.collinear.ai/steer_bare', messages, {"impatience": "low"})

{'role': 'assistant',
 'content': "Tell me exactly how I'm feeling, including any fatigue and pain levels. Get it right, I'm not in the mood for vague descriptions."}